In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests

# Import the API key.
from config import g_key
gmaps.configure(api_key=g_key)

In [2]:
# Store the CSV you saved create in part one into a DataFrame.
city_data_df = pd.read_csv('weather_data/cities.csv')

# Confirm the data dtypes for the data columns
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [3]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []

for temp in max_temp:
    temps.append(max(temp,0))

In [4]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]

# Get the maximum temperature
max_temp = city_data_df["Max Temp"]

# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp], dissipating=False, max_intensity=300, point_radius=4)

# Add the heatmap layer.
fig.add_layer(heat_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
# Heatmap of percent cloudiness
locations = city_data_df[["Lat", "Lng"]]
clouds = city_data_df["Cloudiness"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=clouds, dissipating=False, max_intensity=300, point_radius=4)

# Call the figure to plot the data
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# Heatmap of percent wind speed
locations = city_data_df[["Lat", "Lng"]]
wind = city_data_df["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind, dissipating=False, max_intensity=300, point_radius=4)

# Call the figure to plot the data
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [9]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
5,5,Banda Aceh,ID,2022-08-02 19:51:43,5.5577,95.3222,77.79,79,100,14.79
6,6,Vaini,TO,2022-08-02 19:52:56,-21.2000,-175.2000,75.36,94,40,9.22
10,10,Kongolo,CD,2022-08-02 19:52:57,-5.3833,27.0000,82.08,35,80,3.02
14,14,Jablah,SY,2022-08-02 19:52:59,35.3614,35.9256,80.89,78,40,5.75
21,21,Albany,US,2022-08-02 19:53:02,42.6001,-73.9662,87.78,55,7,3.00
23,23,Hilo,US,2022-08-02 19:53:03,19.7297,-155.0900,81.28,94,100,5.75
29,29,Chifeng,CN,2022-08-02 19:53:05,42.2683,118.9636,75.07,72,100,4.94
30,30,Kapaa,US,2022-08-02 19:52:28,22.0752,-159.3190,81.48,70,20,11.50
34,34,Jamestown,US,2022-08-02 19:53:07,42.0970,-79.2353,77.52,53,75,14.97
36,36,Praia Da Vitoria,PT,2022-08-02 19:53:08,38.7333,-27.0667,79.05,73,40,9.40


In [10]:
preferred_cities_df.count()

City_ID       204
City          204
Country       204
Date          204
Lat           204
Lng           204
Max Temp      204
Humidity      204
Cloudiness    204
Wind Speed    204
dtype: int64

In [14]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""

hotel_df

,City,Country,Max Temp,Lat,Lng,Hotel Name
5,Banda Aceh,ID,77.79,5.5577,95.3222,
6,Vaini,TO,75.36,-21.2000,-175.2000,
10,Kongolo,CD,82.08,-5.3833,27.0000,
14,Jablah,SY,80.89,35.3614,35.9256,
21,Albany,US,87.78,42.6001,-73.9662,
...,...,...,...,...,...,...
572,Bhimunipatnam,IN,81.91,17.8833,83.4333,
575,Manicore,BR,85.14,-5.8092,-61.3003,
577,Manaus,BR,88.30,-3.1019,-60.0250,
583,Potrero Del Llano,MX,87.42,21.0500,-97.6833,


In [12]:
# Set parameters to search for a hotel.
params = {"radius": 5000, "type": "lodging", "key": g_key}

In [17]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location ket for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "lodgubg" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        
hotel_df.head(10)

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Lat,Lng,Hotel Name
5,Banda Aceh,ID,77.79,5.5577,95.3222,OYO 884 Rumoh PMI Hotel
6,Vaini,TO,75.36,-21.2000,-175.2000,Keleti Beach Resort
10,Kongolo,CD,82.08,-5.3833,27.0000,GUEST HOUSE OASIS
14,Jablah,SY,80.89,35.3614,35.9256,هوتيل
21,Albany,US,87.78,42.6001,-73.9662,
23,Hilo,US,81.28,19.7297,-155.0900,Hilo Hawaiian Hotel
29,Chifeng,CN,75.07,42.2683,118.9636,Junyu Tiancheng Hotel
30,Kapaa,US,81.48,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
34,Jamestown,US,77.52,42.0970,-79.2353,Hampton Inn & Suites Jamestown
36,Praia Da Vitoria,PT,79.05,38.7333,-27.0667,Hotel Branco I


In [18]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [19]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [20]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dt>
"""

In [22]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [23]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))